# Single AI crypto concept

In [1]:
from xdata_config import *

/UltimeTradingBot/Crypto_backtest_tools


## 1 - Imports and fonctions

In [2]:
from functions_module import *

/usr/local/lib/python3.9/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/local/lib/python3.9/dist-packages/pandas/core/arrays/masked.py:59: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (
2022-11-12 14:47:27.825262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-12 14:47:27.825301: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-12 14:47:27.864962: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for pl

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import load_model

In [4]:
# chking import
MetaData

,0,Pair,launch_week_stamp,launch_day_stamp,launch_minute
0,BTCUSDT,BTC/USDT,1502668800000,1502928000000,2017-08-17 04:00:00
1,LUNAUSDT,LUNA/USDT,1597622400000,1597968000000,2020-08-21 10:00:00
2,ETHUSDT,ETH/USDT,1502668800000,1502928000000,2017-08-17 04:00:00
3,GMTUSDT,GMT/USDT,1646611200000,1646784000000,2022-03-09 12:00:00
4,USTUSDT,UST/USDT,1639958400000,1640304000000,2021-12-24 08:00:00
...,...,...,...,...,...
106,FIDAUSDT,FIDA/USDT,1632700800000,1632960000000,2021-09-30 12:00:00
107,XNOUSDT,XNO/USDT,1642982400000,1643328000000,2022-01-28 08:00:00
108,BTGUSDT,BTG/USDT,1618185600000,1618531200000,2021-04-16 07:00:00
109,GHSTUSDT,GHST/USDT,1629072000000,1629417600000,2021-08-20 10:00:00


In [5]:
# Costumaize buy condition here
def buy_up_only(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    try:
        
        ## test param
        #buy_pourcent=1
        #window=15
        print (f"---buy_only--- Buy percent: {buy_pourcent}%")
        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b"]=((('
        #codep1='df["buy"]=((('
        for i in range(1,window):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        
        codep1='df["sell"]=((( '
        for i in range(1,window):
            codep1=codep1+'df["low"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo ) | (('
        codep2='df["low"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        df["buy"]=((df['b']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
        
        

    except Exception as e:
        print("Error buy only")
        print(e)
    try:df.pop("b")
    except:print("---buy_only--- no b")
    try:df.pop("sell")
    except:print("---buy_only--- no sell")

    return df

In [6]:
# Costumaize buy condition here
def buy_up_only(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    try:
        
        ## test param
        #buy_pourcent=1
        #window=15
        print (f"---buy_up_only--- Buy percent: {buy_pourcent}%")
        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b"]=((('
        #codep1='df["buy"]=((('
        for i in range(1,window):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        
        codep1='df["sell"]=((( '
        for i in range(1,window):
            codep1=codep1+'df["low"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo ) | (('
        codep2='df["low"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        df["buy"]=((df['b']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
        
        

    except Exception as e:
        print("Error buy only")
        print(e)
    try:df.pop("b")
    except:print("---buy_only--- no b")
    try:df.pop("sell")
    except:print("---buy_only--- no sell")

    return df

In [7]:
# Costumaize buy condition here
def buy_up(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    try:
        
        print (f"---buy_simple_up--- Buy percent: {buy_pourcent}%")
        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["buy"]=((('
        for i in range(1,window):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

    except Exception as e:
        print("Error buy only")
        print(e)
    try:df.pop("b")
    except:print("---buy_only--- no b")
    try:df.pop("sell")
    except:print("---buy_only--- no sell")

    return df

In [8]:
# Costumaize buy condition here
def buy_min_up(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    try:
        
        print (f"---buy_min_up--- Buy percent: {buy_pourcent}%")
        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["buy"]=((('
        for i in range(1,window):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        df['ismin'] = np.where(
        df['high'] <= df.shift(-window-1).rolling(2*window)['high'].min(), 1,
       0
        )
        df["buy"]=((df['buy']==1 ) & (df['ismin']==1)).replace({False: 0, True: 1})

    except Exception as e:
        print("Error buy only")
        print(e)
    try:df.pop("b")
    except:print("---buy_only--- no b")
    try:df.pop("ismin")
    except:print("---buy_only--- no sell")

    return df

In [9]:
buy_only=buy_min_up

In [10]:
def mini_expand3(pair="GMT/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=1,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='1 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='5 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list15m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='15 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1h["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 H'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1d["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 d'):Pair_Full.iloc[len(Pair_Full)-1].name],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    # Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
    #         right_index=True, suffixes=('', ''))
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='inner',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    #buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    buy_only(Merged,buy_pourcent=buy_pourcent,sell_pourcent=sell_pourcent,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    # Merged=justlast_remover(Merged)
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    Merged=Merged.dropna()
    return Merged

## 2- Garbage Test

In [11]:
# for pair in Binance_USDT_HALAL:
#     price_volatility_15m=(100*(df_list15m[pair]["high"]-df_list15m[pair]["low"])/df_list15m[pair]["high"]).mean()

In [12]:
pair="GMT/USDT"
price_volatility_15m=(100*(df_list15m[pair]["high"]-df_list15m[pair]["low"])/df_list15m[pair]["high"]).mean()
print(price_volatility_15m)

2.173600867703185


In [13]:
df=mini_expand3(pair=pair,i=0,j=len(df_list1m[pair]),window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=max(price_volatility_15m*0.5,0.5),sell_pourcent=SELL_PERCENT)
print("df original shape "+str(df.shape))
print(f"df original shape buy mean : {df.buy.mean()*100}")
df=df.reset_index()
try:df.pop("num_index")
except: pass
try:df.pop("index")
except: pass
try:df.pop("date")
except: pass
df=data_shufler(df)            
df=data_chooser(df,weight=50,row_numbers=df.buy.sum()*2)
gc.collect()
df=data_cleanup(df)
df=df.dropna()
#df.pop("price");print("we work with no price")
#df.pop("BTC_price")

---buy_min_up--- Buy percent: 1.0868004338515924%
---buy_only--- no b
df original shape (80041, 607)
df original shape buy mean : 1.439262378031259


In [14]:
df

,price,high-1,low-1,close-1,volume-1,high-2,low-2,close-2,volume-2,high-3,...,BTC_volume-14_5min,BTC_high-15_5min,BTC_low-15_5min,BTC_close-15_5min,BTC_volume-15_5min,day,hour,minute,lunch_day,buy
num_index,,,,,,,,,,,,,,,,,,,,,
212,0.782490,-0.002760,-0.000166,-0.000997,168277.0,-0.003195,0.001470,-0.000166,207669.9,-0.003208,...,43.10983,0.001885,0.002908,0.002819,101.45701,1,5,58,-798,1
1941,1.517343,-0.004625,0.003442,-0.000578,165331.7,-0.004954,0.003172,0.002137,238685.8,-0.021859,...,138.17574,0.015345,0.018465,0.017885,107.13211,6,23,44,-798,0
1562,2.528493,-0.002186,-0.000201,-0.000323,221794.1,-0.000924,0.000408,-0.000869,155515.1,-0.002123,...,186.60728,-0.001226,0.000405,-0.001225,95.04365,7,10,23,-798,0
928,3.814600,-0.003070,0.002726,0.000786,284839.6,-0.002404,0.003631,0.001130,468066.9,-0.007214,...,223.66552,-0.004182,-0.001203,-0.001984,214.58472,3,18,4,-798,1
533,2.247600,-0.006807,-0.001068,-0.003844,345663.6,-0.004921,-0.001068,-0.003964,331741.7,-0.008187,...,199.13739,-0.007834,-0.003855,-0.005641,381.18712,1,13,29,-798,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1.131805,-0.004590,0.000702,-0.002266,1113649.8,-0.005465,-0.000181,-0.002346,1517544.3,-0.005474,...,96.38421,0.005409,0.007182,0.005802,110.22162,2,9,15,-798,1
695,2.181018,-0.003949,0.002466,0.001544,288436.1,-0.006934,-0.002152,-0.003169,137456.6,-0.005645,...,133.64638,-0.008863,-0.005994,-0.007959,274.39408,2,2,27,-798,1
1435,3.360700,-0.004999,0.006320,-0.000839,991189.0,0.003184,0.007617,0.004725,339620.1,0.006011,...,109.98166,0.003102,0.006736,0.005614,251.02598,7,15,49,-798,0


In [15]:
print("df choosen data shape"+str(df.shape))
(df.shape[0]/2)==df.buy.sum()

df choosen data shape(2304, 607)


True

In [16]:
dt=df.to_numpy(dtype=np.float32)

In [17]:
dt[0,2]==df.iloc[0,2]

False

In [18]:
index_20percent= int(0.2*len(dt[:,0]))
print(index_20percent)
XVALIDATION= dt[:index_20percent, :-1]
YVALIDATION= dt[:index_20percent,-1]
XTRAIN= dt[index_20percent:, 0:-1]
YTRAIN= dt[index_20percent:,-1]

460


In [19]:
XVALIDATION[0,2]

-0.0001661363

In [20]:

if True:
    if FIRST_NORM_FLAG:
    #if True:
        print("normalizing ...")
        mean = XTRAIN.mean(axis=0)
        std = XTRAIN.std(axis=0)



        XTRAIN -= mean 
        XTRAIN /= std

        XVALIDATION -=mean
        XVALIDATION /= std
        FIRST_NORM_FLAG=False
        ######################### SAVIN NORM ################
        try:
            Normalization={"mean":mean.tolist(),"std":std.tolist()}
            with open(Normalization_File.replace("/UltimeTradingBot/Data/", "/UltimeTradingBot/Data/"+pair.replace("/", "-")+"-"), 'w+') as fp:
                        json.dump(Normalization, fp,  indent=4)
                        print(Normalization_File)
        except Exception as e:
            print("error Normalization in juppiter")
            print(e)
    else:print("already normalized")

normalizing ...
/UltimeTradingBot/Data/tp77_w15_max5min_Norm_v1.json


In [21]:
Y=dt[:,-1]

In [22]:
IN_DIM=dt.shape[1]-1
model = Sequential()
model.add(Dense(int(IN_DIM),input_dim=IN_DIM,activation='relu')) #( 100=>66.23)
# resultus withe 250 d0.3 50 d 20 d 8 = 65.6% vs  65.49 no droppout vs 65.78 d0.5 vs 65.68 d0.4 # 65.48 # one dropout of   0.5 = 66.11   #tanh 66.33 accuracy #softmax 66.12 #softplus 66.5 # sigmoid 66.21 / 
#model.add(Dropout(0.1))
model.add(Dense(int(IN_DIM),activation='softplus')) # best softplus 66.26(69) vs relu 66.65(70)  / -20 => 66.25 /250 ->65.78
#model.add(Dropout(0.1))
model.add(Dense(int(IN_DIM),activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(int(IN_DIM/2),activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(int(IN_DIM/2),activation='relu'))
#model2.add(Dropout(0.1)) #5=> 66.00  66.21 vs no dopout 66.28
#model2.add(Dense(int(20),activation='relu')) # disabled 6.24
model.add(Dense(int(50),activation='relu')) # -4 -> -> 66.24
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath =MODEL_FILE+f"test.multi_train.h5",monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=15,verbose=1)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:,-1],
                validation_data=(dt[:index_20percent, :-1],Y[:index_20percent]),
                epochs=6000,
                batch_size=1000,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#94%

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 606)               367842    
                                                                 
 dense_1 (Dense)             (None, 606)               367842    
                                                                 
 dense_2 (Dense)             (None, 606)               367842    
                                                                 
 dense_3 (Dense)             (None, 303)               183921    
                                                                 
 dense_4 (Dense)             (None, 303)               92112     
                                                                 
 dense_5 (Dense)             (None, 50)                15200     
                                                                 
 dense_6 (Dense)             (None, 1)                 5

2022-11-12 14:49:00.513441: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-12 14:49:00.513478: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-12 14:49:00.513504: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: abj-K93SV
2022-11-12 14:49:00.513514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: abj-K93SV
2022-11-12 14:49:00.513574: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2022-11-12 14:49:00.513619: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 390.154.0


None
Epoch 1/6000
2/2 [==============================] - 1s 549ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 2/6000
2/2 [==============================] - 0s 137ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 3/6000
2/2 [==============================] - 0s 149ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 4/6000
2/2 [==============================] - 0s 155ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 5/6000
2/2 [==============================] - 0s 142ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 6/6000
2/2 [==============================] - 0s 146ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 7/6000
2/2 [==============================] - 0s 140ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5370
Epoch 8/6000
2/2 [=========================